In [239]:
import pandas as pd, numpy as np, json, os

In [240]:
# os.chdir('E:/Onedrive - Lancaster University/datarepo/influence/ro')
os.chdir('C:/users/csala/Onedrive - Lancaster University/datarepo/influence/ro')

In [241]:
nmembers=json.loads(open('data/nmembers.json','r').read())

In [242]:
img=json.loads(open('data/img.json','r').read())

In [243]:
deaths=json.loads(open('data/deaths.json','r').read())

In [244]:
# os.chdir('E:/Skydrive/Github/influence/ro')
os.chdir('C:/users/csala/Onedrive/Github/influence/ro')

In [245]:
legs={'2016':['2016-12-21','2019-07-01'],
'2012':['2012-12-20','2016-12-20'],
'2008':['2008-12-19','2012-12-19'],
'2004':['2004-12-19','2008-12-13'],
'2000':['2000-12-15','2004-11-30'],
'1996':['1996-11-27','2000-11-30'],
'1992':['1992-10-28','1996-11-22'],
'1990':['1990-06-19','1992-10-16']}

In [246]:
import altair as alt

Members

In [247]:
szen=['🔼 Szenátus','🔽 Képviselőház']
szen2=['Senat','Camera Deputaților']

In [248]:
istart=legs['1990'][0]
iend=legs['2016'][1]

In [249]:
ro_months={'ian.':'Jan',
          ' ia ':' Jan ',
          'feb.':'Feb',
          'mar.':'Mar',
          'apr.':'Apr',
          'mai':'May',
          'iun.':'Jun',
          ' iu ':' Jun ',
          'iul.':'Jul',
          'aug.':'Aug',
          'sep.':'Sep',
          'oct.':'Oct',
          'noi.':'Nov',
          'dec.':'Dec',
          'ianuarie':'Jan',
          'februarie':'Feb',
          'martie':'Mar',
          'aprilie':'Apr',
          'mai':'May',
          'iunie':'Jun',
          'iulie':'Jul',
          'august':'Aug',
          'septembrie':'Sep',
          'octombrie':'Oct',
          'noiembrie':'Nov',
          'decembrie':'Dec'}
def date_ro(d,s=True):
    if 'prezent' in d:
        t=pd.to_datetime(legs['2016'][1])
    else:
        ds=d.split(' ')
        x=ds[1].replace('0201','2001').replace('0092','1992')
        t=pd.to_datetime(ro_months[ds[0].strip()]+' '+x)
    if s: return str(t)[:10]
    else: return t

In [250]:
def replace_all(text, dic):
    for i in dic:
        text = text.replace(i, dic[i])
    return text

In [251]:
m=nmembers['Antonescu George Crin Laurentiu | 1959-09-21']
data=[]
for p in m['Parties']:
    start=pd.to_datetime(p['start'])
    end=pd.to_datetime(p['end'])    
    d={}
    d['Name']=m['Name']['full']
    d['Csoportok']=p['party']
    d['Típus']='Párt'
    d['start']=start
    d['end']=end
    d['duration']=pd.to_timedelta(end-start).days
    data.append(d)
    d=dict(d)
    d['Csoportok']=p['judet']
    d['Típus']='Megye'
    data.append(d)
for p in m['Countries']:
    start=pd.to_datetime(p['start'])
    end=pd.to_datetime(p['end'])    
    d={}
    d['Name']=m['Name']['full']
    d['Csoportok']=p['country']
    d['Típus']='Ország'
    d['start']=start
    d['end']=end
    d['duration']=pd.to_timedelta(end-start).days
    data.append(d)
for p in m['Groups']:
    start=pd.to_datetime(p['start'])
    end=pd.to_datetime(p['end'])    
    d={}
    d['Name']=m['Name']['full']
    d['Csoportok']=p['group'].replace('Comisia pentru','').replace('Comisia','').strip().capitalize()
    d['Típus']='Bizottság'
    d['start']=start
    d['end']=end
    d['duration']=pd.to_timedelta(end-start).days
    data.append(d)
for pi,p in enumerate(m['Link']):
    start=pd.to_datetime(legs[p[-10:-6]][0])
    end=pd.to_datetime(legs[p[-10:-6]][1])    
    d={}
    d['Name']=m['Name']['full']
    d['Típus']='Kamara'
    kamara=m['Camera'][pi]
    pres=''
    if '#' in kamara:
        pres='#'+kamara.split('#')[1]+'#'
        if ('din ') in kamara:
            b=dict(d)
            b['Csoportok']=szen2[int(p[-1])-1]
            b['start']=start
            start=max(start,date_ro(kamara[kamara.find('din ')+4:].strip(),False))
            b['end']=start
            b['duration']=pd.to_timedelta(end-start).days
            data.append(b)
        if ('până în ') in kamara:
            b=dict(d)
            b['Csoportok']=szen2[int(p[-1])-1]
            b['end']=end
            end=min(end,date_ro(kamara[kamara.find('până în ')+8:].strip(),False))
            b['start']=end
            b['duration']=pd.to_timedelta(end-start).days
            data.append(b)
    d['Csoportok']=pres+szen2[int(p[-1])-1]
    d['start']=start
    d['end']=end
    d['duration']=pd.to_timedelta(end-start).days
    data.append(d)
ef=pd.DataFrame(data)
eord=ef.groupby('Típus').sum()
eord['duration']=[1,60,30,4,50]
eord.columns=['order']
ef=ef.set_index('Típus').join(eord).reset_index()

In [252]:
alt.Chart(ef,
          title=m['Name']['full']
         ).mark_bar().encode(
    x=alt.X('start:T',title=''),
    x2='end:T',
    y=alt.Y('Csoportok:N',title='',
            sort=alt.EncodingSortField(field="order", op="mean", order='descending')),
    color=alt.Color('Típus:N',
        sort=alt.EncodingSortField(field="order", op="mean", order='descending'))
)

<VegaLite 3 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


Save JSON

In [253]:
def party_normalizer(party):
    if party in ['N/A','NA','NI',['NA', 'NI'],'-','Independent','Minorități',
                'Fără adeziune la formaţiunea politică pentru care a candidat la alegeri']: return '👤'
    elif party in ['ALDE RO','ALDE Romania','Partidul Conservator','Liberal Conservator',
                   'PC','Partidul Puterii Umaniste','ALDE','PUNR','PUR','PUR-SL']: return '🕊️'
    elif party in ['Forumul Democrat al Germanitor din România','FDGR']: return '⚫️'
    elif party in ['Partidul Naţional Liberal','PNL-CD',"PL'93",'PNL-AT','PAC','FC - Forţa Civică',
        'Partidul Liberal Democrat','PNL','Partida Naţională','Naţional Liberal','Alianţei Civice']: return'📐'
    elif party in ['Partidul Democrat','Partidul Democrat-Liberal','Democrat','Democrat Liberal','Democrat-Liberal în feb-2008',
                   'Democrat-Liberal','Democrat care se transforma în','PD care se transforma în','PDL în feb-2008','Dreptate şi Adevăr PNL-PD',
        'PD-L','PD','PDL']: return'🔶'
    elif party in ['Partidul Mișcarea Populară','PMP']: return '🍏'
    elif party in ['Partidul Naţional Ţaranesc Creştin Democrat','PNȚCD','PNTCD']:return '✳️'
    elif party in ['Partidul România Mare','PRM']:return '🔱'
    elif party in ['FSN','FDSN']:return '⚪️'
    elif party in ['USR']:return '🔷'
    elif party in ['PP-DD','Democrat Popular']:return '⚛️'
    elif party in ['PER','MER','PDAR','FER']:return '🍀'
    elif party in ['PSD','Partidul Social Democrat','Partidul Social Democrat + Partidul Conservator',
                   'PDSR','PSDR','PSM','P.S.D.','Uniunea PSDă -  PSD Român',
                 'democrat-agrar şi social-democrat',
                 'ecologist şi social-democrat',
                 'social-democrat al  PSD Român']:return '🌹'
    elif party in ['Romániai Magyar Demokrata Szövetség',
        'Uniunea Democrată Maghiară din România','RMDSZ','UDMR']:return '🌷'
    elif party in ['PRO Europa']: return '🧑'
    elif party in ['Uniunea Națională pentru Progresul României','UNPR']: return '🦅'
    else: 
        return '🔘'

In [254]:
def group_party_normalizer(party):
    party0=party[:party.rfind('#')+1]
    party=party[party.rfind('#')+1:]
    party=replace_all(party,{'Grupul parlamentaral ':'',
                        'rtidului':'rtidul','fost':'',
                'Partidul Democrat Liberal':'PDL','Partidul Democrat-Liberal':'PDL',
                    'Partidul Democrat-liberal':'PDL'}).strip()
    party=replace_all(party,{'Grupul parlamentar':'','Partidul Democrat':'PD',
                            'Partidul':'','Liberal Conservator':'PC',
                            }).strip()
    if '(' in party:
        party=party.split('(')[0].strip()
    group_party_dict={'"Partida Naţională"':'Partida Naţională',
                        'Frontului Salvării Naţionale':'FSN',
                     'P.S.D.':'PSD','Conservator':'PC',
                      'Democraţiei Sociale din România':'PSD',
                      'PD care se transforma în':'PD',
                      'PDL în feb-2008':'PDL',
                      'Liberal 1993 si al  Aliantei Civice':'Alianţei Civice',
                      'Naţional Creştin Democrat':'PNTCD',
                      'Naţional Liberal':'PNL',
                      'socialist':'PSD',
                      'Naţional Ţărănesc - creştin şi democrat':'PNTCD',
                      'Naţional Ţărănesc Creştin Democrat Civic-Ecologist':'PNTCD',
                      'Naţional Ţărănesc Creştin Democrat şi al  Ecologist Român':'PNTCD',
                      'ţi neafiliaţi':'Independent',
                      'Uniunii Democrate Maghiare din România':'RMDSZ',
                      'Poporului - Dan Diaconescu':'PP-DD',
                      'România Mare':'PRM',
                      'Social Democrat':'PSD',
                      'Uniunea PSDă - PD':'PSD',
                      'Umanist din România':'PUR',
                      't neafiliat':'Independent',
                      'neafiliat':'Independent',
                      'minorităţilor naţionale':'Minorități',
                      'Uniunii Naţionale pentru Progresul României':'UNPR',
                      'Alianţei Civice':'PAC',
                      'de orientare civic-liberală':'PAC',
                      'Uniunii Salvaţi România':'USR',
                      'Partida Naţională':'PNL',
                      'Mişcarea Populară':'PMP',
                      'PD Agrar din România':'PDAR',
                      'PD în feb-2008':'PD',
                      'Unităţii Naţionale Române':'PUNR',
                      'Unităţii Naţionale a Românilor':'PUNR',
                      'deputaţilor independenţi':'Independent',
                      'i neafiliaţi':'Independent',
                      'naţional democrat':'PNTCD',
                      'progresist':'PC',
                      'Democrat Popular':'PP-DD',
                     'Dreptate şi Adevăr PNL-PD':'PDL',
                     'Uniunea PSDă -  PSD Român':'PSD',
                     'democrat-agrar şi social-democrat':'PSD',
                     'ecologist şi social-democrat':'PSD',
                     'social-democrat al  PSD Român':'PSD'
                     }
    party=party_normalizer(replace_all(party,
                group_party_dict))+' '+\
                replace_all(party,group_party_dict)
    return party0+party

In [255]:
def party_writer(party):
    if party in ['Romániai Magyar Demokrata Szövetség',
        'Uniunea Democrată Maghiară din România','RMDSZ','UDMR']:return 'RMDSZ'
    elif party in ['Uniunea PSDă -  PSD Român',
 'democrat-agrar şi social-democrat',
 'ecologist şi social-democrat',
 'social-democrat al  PSD Român']:
        return 'PSD'
    elif party in ['Fără adeziune la formaţiunea politică pentru care a candidat la alegeri']:
        return 'Independent'
    else: 
        return party

In [256]:
# megyek=json.loads(open('E:/Skydrive/Github/universal/ro/hun_megyek.json','r').read())
megyek=json.loads(open('C:/users/csala/Onedrive/Github/universal/ro/hun_megyek.json','r').read())
megyek['La nivel national']='Országos'

In [257]:
def extender(r,n=5):
    p={}
    for i in r:
        for j in range(n):
            p[i+j*' ']=r[i]+j*' '
    return p

In [258]:
cc={"":"uu",
"/camera_deputatilor/grupuri_prietenie/flags1/africadesud.gif":"za",
"/camera_deputatilor/grupuri_prietenie/flags1/albania.gif":"al",
"/camera_deputatilor/grupuri_prietenie/flags1/algeria.gif":"dz",
"/camera_deputatilor/grupuri_prietenie/flags1/arabiasaudi.gif":"sa",
"/camera_deputatilor/grupuri_prietenie/flags1/argentina.gif":"ar",
"/camera_deputatilor/grupuri_prietenie/flags1/armenia.gif":"am",
"/camera_deputatilor/grupuri_prietenie/flags1/australia.gif":"au",
"/camera_deputatilor/grupuri_prietenie/flags1/austria.gif":"at",
"/camera_deputatilor/grupuri_prietenie/flags1/azerbaidjan.gif":"az",
"/camera_deputatilor/grupuri_prietenie/flags1/bahrein.gif":"bh",
"/camera_deputatilor/grupuri_prietenie/flags1/belarus.gif":"by",
"/camera_deputatilor/grupuri_prietenie/flags1/belgia.gif":"be",
"/camera_deputatilor/grupuri_prietenie/flags1/bolivia.gif":"bo",
"/camera_deputatilor/grupuri_prietenie/flags1/bosnia.gif":"ba",
"/camera_deputatilor/grupuri_prietenie/flags1/brazilia.gif":"br",
"/camera_deputatilor/grupuri_prietenie/flags1/bulgaria.gif":"bg",
"/camera_deputatilor/grupuri_prietenie/flags1/canada.gif":"ca",
"/camera_deputatilor/grupuri_prietenie/flags1/cehia.gif":"cz",
"/camera_deputatilor/grupuri_prietenie/flags1/chile.gif":"cl",
"/camera_deputatilor/grupuri_prietenie/flags1/china.gif":"cn",
"/camera_deputatilor/grupuri_prietenie/flags1/cipru.gif":"cy",
"/camera_deputatilor/grupuri_prietenie/flags1/columbia.gif":"co",
"/camera_deputatilor/grupuri_prietenie/flags1/congo.gif":"cg",
"/camera_deputatilor/grupuri_prietenie/flags1/coreea.gif":"kr",
"/camera_deputatilor/grupuri_prietenie/flags1/costarica.gif":"cr",
"/camera_deputatilor/grupuri_prietenie/flags1/cotedivoir.gif":"ci",
"/camera_deputatilor/grupuri_prietenie/flags1/croatia.gif":"hr",
"/camera_deputatilor/grupuri_prietenie/flags1/cuba.gif":"cu",
"/camera_deputatilor/grupuri_prietenie/flags1/danemarca.gif":"dk",
"/camera_deputatilor/grupuri_prietenie/flags1/ecuador.gif":"ec",
"/camera_deputatilor/grupuri_prietenie/flags1/egipt.gif":"eg",
"/camera_deputatilor/grupuri_prietenie/flags1/elvetia.gif":"ch",
"/camera_deputatilor/grupuri_prietenie/flags1/emirate.png":"ae",
"/camera_deputatilor/grupuri_prietenie/flags1/estonia.gif":"ee",
"/camera_deputatilor/grupuri_prietenie/flags1/filipine.gif":"ph",
"/camera_deputatilor/grupuri_prietenie/flags1/finlanda.gif":"fi",
"/camera_deputatilor/grupuri_prietenie/flags1/franta.gif":"fr",
"/camera_deputatilor/grupuri_prietenie/flags1/georgia.gif":"ge",
"/camera_deputatilor/grupuri_prietenie/flags1/germania.gif":"de",
"/camera_deputatilor/grupuri_prietenie/flags1/grecia.gif":"gr",
"/camera_deputatilor/grupuri_prietenie/flags1/guineea.gif":"gn",
"/camera_deputatilor/grupuri_prietenie/flags1/india.gif":"in",
"/camera_deputatilor/grupuri_prietenie/flags1/indonezia.gif":"id",
"/camera_deputatilor/grupuri_prietenie/flags1/iordania.gif":"jo",
"/camera_deputatilor/grupuri_prietenie/flags1/irak.gif":"iq",
"/camera_deputatilor/grupuri_prietenie/flags1/iran.gif":"ir",
"/camera_deputatilor/grupuri_prietenie/flags1/irlanda.gif":"ie",
"/camera_deputatilor/grupuri_prietenie/flags1/islanda.gif":"is",
"/camera_deputatilor/grupuri_prietenie/flags1/israel.gif":"il",
"/camera_deputatilor/grupuri_prietenie/flags1/italia.gif":"it",
"/camera_deputatilor/grupuri_prietenie/flags1/iugoslavia.gif":"uu",
"/camera_deputatilor/grupuri_prietenie/flags1/japonia.gif":"jp",
"/camera_deputatilor/grupuri_prietenie/flags1/kazahtan.gif":"kz",
"/camera_deputatilor/grupuri_prietenie/flags1/kuwait.gif":"kw",
"/camera_deputatilor/grupuri_prietenie/flags1/letonia.gif":"lv",
"/camera_deputatilor/grupuri_prietenie/flags1/liban.gif":"lb",
"/camera_deputatilor/grupuri_prietenie/flags1/lituania.gif":"lt",
"/camera_deputatilor/grupuri_prietenie/flags1/luxemburg.gif":"lu",
"/camera_deputatilor/grupuri_prietenie/flags1/macedonia.gif":"mk",
"/camera_deputatilor/grupuri_prietenie/flags1/malaezia.gif":"my",
"/camera_deputatilor/grupuri_prietenie/flags1/malta.gif":"mt",
"/camera_deputatilor/grupuri_prietenie/flags1/mareabritanie.gif":"gb",
"/camera_deputatilor/grupuri_prietenie/flags1/maroc.gif":"ma",
"/camera_deputatilor/grupuri_prietenie/flags1/mexic.gif":"mx",
"/camera_deputatilor/grupuri_prietenie/flags1/mongolia.gif":"mn",
"/camera_deputatilor/grupuri_prietenie/flags1/muntenegru.gif":"me",
"/camera_deputatilor/grupuri_prietenie/flags1/norvegia.gif":"no",
"/camera_deputatilor/grupuri_prietenie/flags1/olanda.gif":"nl",
"/camera_deputatilor/grupuri_prietenie/flags1/oman.gif":"om",
"/camera_deputatilor/grupuri_prietenie/flags1/pakistan.gif":"pk",
"/camera_deputatilor/grupuri_prietenie/flags1/panama.gif":"pa",
"/camera_deputatilor/grupuri_prietenie/flags1/peru.gif":"pe",
"/camera_deputatilor/grupuri_prietenie/flags1/polonia.gif":"pl",
"/camera_deputatilor/grupuri_prietenie/flags1/portugalia.gif":"pt",
"/camera_deputatilor/grupuri_prietenie/flags1/qatar.gif":"qa",
"/camera_deputatilor/grupuri_prietenie/flags1/rusia.gif":"ru",
"/camera_deputatilor/grupuri_prietenie/flags1/senegal.gif":"sn",
"/camera_deputatilor/grupuri_prietenie/flags1/singapore.gif":"sg",
"/camera_deputatilor/grupuri_prietenie/flags1/siria.gif":"sy",
"/camera_deputatilor/grupuri_prietenie/flags1/slovacia.gif":"sk",
"/camera_deputatilor/grupuri_prietenie/flags1/slovenia.gif":"si",
"/camera_deputatilor/grupuri_prietenie/flags1/spania.gif":"es",
"/camera_deputatilor/grupuri_prietenie/flags1/srilanka.gif":"lk",
"/camera_deputatilor/grupuri_prietenie/flags1/sudan.gif":"sd",
"/camera_deputatilor/grupuri_prietenie/flags1/suedia.gif":"se",
"/camera_deputatilor/grupuri_prietenie/flags1/thailanda.gif":"th",
"/camera_deputatilor/grupuri_prietenie/flags1/tunisia.gif":"tn",
"/camera_deputatilor/grupuri_prietenie/flags1/turcia.gif":"tr",
"/camera_deputatilor/grupuri_prietenie/flags1/turkmenistan.gif":"tm",
"/camera_deputatilor/grupuri_prietenie/flags1/ucraina.gif":"ua",
"/camera_deputatilor/grupuri_prietenie/flags1/ungaria.gif":"hu",
"/camera_deputatilor/grupuri_prietenie/flags1/urss.gif":"uu",
"/camera_deputatilor/grupuri_prietenie/flags1/usa.png":"us",
"/camera_deputatilor/grupuri_prietenie/flags1/uruguay.gif":"uy",
"/camera_deputatilor/grupuri_prietenie/flags1/venezuela.gif":"ve",
"/camera_deputatilor/grupuri_prietenie/flags1/vietnam.gif":"vn",
"/camera_deputatilor/grupuri_prietenie/flags1/zambia.gif":"zm"}

In [259]:
cr={'Delegaţia Parlamentului României la Adunarea Parlamentară a Francofoniei':'Frankofónia',
   'Grupul parlamentar de prietenie cu UNESCO':'🇺🇳 UNESCO',
    'Grupul parlamentar de prietenie cu Republica Franceză - Adunarea Naţională':'🇫🇷 Franciaország',
    'Grupul parlamentar de prietenie cu Republica Franceză - Senat':'🇫🇷 Franciaország',
    'Grupul parlamentar de prietenie cu Republica Serbia':'🇷🇸 Szerbia',
    'Delegaţia Camerei Deputaţilor la Adunarea Interparlamentară a Ortodoxiei':'Ortodoxia',
    'Delegaţia Parlamentului României la Adunarea Parlamentară a Consiliului Europei':'Európa Tanács',
    'Delegaţia Parlamentului României la Adunarea Parlamentară a Cooperării Economice a Mării Negre':'Fekete tenger',
    'Delegaţia Parlamentului României la Adunarea Parlamentară a Mediteranei':'Földközi tenger',
    'Delegaţia Parlamentului României la Adunarea Parlamentară a NATO':'NATO',
    'Delegaţia Parlamentului României la Adunarea Parlamentară a OSCE':'OSCE',
    'Delegaţia Parlamentului României la Adunarea Parlamentară a Uniunii pentru Mediterana':'Földközi tenger',
    'Delegaţia Parlamentului României la Adunarea Uniunii Europei Occidentale':'Kelet-Európa',
    'Delegaţia Parlamentului României la Iniţiativa Central-Europeană - Dimensiunea Parlamentară':'Közép-Európa',
    'Delegaţia permanentă a Parlamentului României la Adunarea Parlamentară a Procesului de Cooperare din Europa de Sud-Est':'Délkelet-Európa',
'Grupul mixt de control parlamentar specializat al Europol':'Europol',
   'Comisia Interparlamentară Bucureşti - Chişinău (partea română)':'🇲🇩 Moldova'}

In [260]:
#!pip install pycountry
import pycountry
import gettext
#!pip install emoji-country-flag
import flag
hun = gettext.translation('iso3166', pycountry.LOCALES_DIR,
                              languages=['hu'])
hun.install()

In [261]:
def local_country(c):
    if c=='uu': return ''
    z=_(pycountry.countries.get(alpha_2=c.upper()).name)
    #manual overrides
    if z=='Koreai Köztársaság':z='Dél-Korea'
    if z=='Nagy-Britannia (Egyesült Királyság)':z='Egyesült Királyság'
    if z=='Macedón Köztársaság':z='Macedónia'
    if z=='Szíriai Arab Köztársaság':z='Szíria'
    if z=='Venezuelai Bolivári Köztársaság':z='Venezuela'
    if z=='Orosz Föderáció':z='Oroszország'
    g=flag.flagize(':'+c.upper()+':')
    return g+' '+z

In [262]:
def tolerance_cleaner(ef,dt=30):
    hfs=[]
    df=ef.sort_values('start').groupby(['Típus','Csoportok','start','end']).first()
    df=df.reset_index(level=[2,3], drop=False)
    for p in df.index.get_level_values(0).unique():
        for r in df.loc[p].index.get_level_values(0).unique():
            hf=df.loc[p].loc[[r]].sort_values('start')
            hf['dt']=list((hf[1:]['start']-hf['end'][:-1]).dt.days.values)+[0]
            hf=hf.reset_index(drop=False)
            new_column=[]
            counters={i:0 for i in hf['Csoportok'].unique()}
            for i in hf.index:
                j=hf.loc[i]['Csoportok']
                new_column.append(j+counters[j]*' ')
                if hf.loc[i]['dt']>dt:
                    counters[j]+=1
            hf['Csoportok1']=new_column
            hf['Típus']=p
            hf=hf.drop(['dt'],axis=1)
            hfs.append(hf)
    hfs=pd.concat(hfs)
    hfs=hfs.reset_index(drop=True)
    return(hfs)

In [263]:
pres_all={'#VP#':'','#PRES#':'','#SECR#':'','#TRES#':'','#SEF#':'','#CHES#':''}

In [279]:
medals={'#INTER#':'🇺🇳 ','#inter#':'🇺🇳 ','#GRUP#':'🎴 ','#BP#':'',
        '#VP#':'🥈','#PRES#':'🥇','#SECR#':'🥉','#TREZ#':'🥉','#PRCV#':'🥉','#SEF#':'🏅','#CHES#':'🏅','#SPEC#':'🚩','#ANCH#':'🏳️',
        '#vp#':'🥈','#pres#':'🥇','#secr#':'🥉','#trez#':'🥉','#prcv#':'🥉','#sef#':'🏅','#ches#':'🏅','#spec#':'🚩','#anch#':'🏳️',
       }
medals_none={i:'' for i in medals}

In [280]:
biz_common={'comisia':'','a camerei deputaţilor şi senatului':'',
          'permanentă':'', 'comună':'', 'specială':'',
            'privind':'','pentru':'','de anchetă':'','parlamentară':'',
            '  ':' ','  ':' ','  ':' ','  ':' ','  ':' ','  ':' '}

In [281]:
biz_flags={"Biztonság":"🛃",
"Egészségügy":"♿️",
"Emberi jogok, kisebbségek":"🚹",
"Európa":"⏺️",
"Parlament":"🅿️",
"Gazdaság, ipar":"⏫",
"Infrastruktúra, energia, kommunikáció":"➿",
"Közigazgatás, környezetvédelem":"🔄",
"Külpolitika, diaszpóra":"🌐",
"Mezőgazdaság, erdők, altalajkincsek":"🚰",
"Munka, szociális és családi ügyek":"🚻",
"Művészet, média":"🎦",
"Oktatás, tudomány, sport":"🔠",
"Pénzügy":"🔢",
"Állambiztonság":"🛂",
'Más':'⏹️',
"Állami ügyek":"🔀",
'Költségvetés':'💹',
'Választások':'✅'}

In [282]:
def biz_cleaner(biz):
    csop=replace_all(biz.strip(),medals_none).lower()
    csop=replace_all(csop,biz_common).strip()
    #medal=replace_all(biz[:biz.rfind('#')+1],medals)
    medal=biz[:biz.rfind('#')+1]
    abbr_in={'Művészet, média':['televiziun','mijloace de informare în mas'],
            'Pénzügy':['finan','bănci'],
            'Költségvetés':['curţii de contur','buget','fost cheltuite sumele','sumele din fondul constituit în cotă'],
            'Állami ügyek':['decembrie 198','corupţ','legii privatizări','suspendare din funcţie a preşedintelui',
                           'centenarului','marii uniri','constituţie','funcţionarea partidelor politic',
                           'privatizare','cod civil','cod penal','funcţionarea justiţiei','abuzurilor şi petiţii',
                           'privatizată','valea jiului','codului penal','codului administrati','decoraţii',
                           '13-15 iunie 199','nicolae ceauşesc','domeniul justiţie'],
            'Mezőgazdaság, erdők, altalajkincsek':['exploatării minereu','agricultură','forestier','irigaţi'],
            'Parlament':['alegerea camerei deputa','adunării deputaţilo','de validar','disciplină','imunităţi',
                        'regulamentului camerei deputa','regulamentului senatulu','statutul deputatului şi senatorulu',
                        'regulamentului şedinţelor comune','regulament','alegerea membrilor parlamentului',
                        ' ale camerei deputaţilor şi senatulu'],
            'Állambiztonság':['serviciului de protecție și pază','siguranţă naţional',
                             'serviciului de informaţii exter','activităţii sr','centre de detenţie ale cia',
                             'securităţii naţional','interceptarea comunicaţiilor','securitatea naţional'],
             'Európa':['europen','schenge','europea'],
             "Közigazgatás, környezetvédelem":["urbanis",'menajarea teritoriulu','administraţie public','mediu',
                                              'cadastru'],
            'Infrastruktúra, energia, kommunikáció':['energ',' tarom','tehnologia informaţiei'
                                                     'comunicaţiilor','transporturi','infrastructură',
                                                    'informaţiei şi comunicaţiilor'],
             "Emberi jogok, kisebbségek":["minorităţ",'demnitate uman','egalitate','antidiscriminare',
                                         'drepturile fundamentale ale omului','minoritatea','minorități','romi',
                                          'persoanelor private de libertate','oportunităţi egale',
                                          'drepturilor persoanelor'],
            "Munka, szociális és családi ügyek":['problemelor economice, sociale','muncă','familie',
                                                 'protecţie socială','protecţia copilului'],
             'Más':['cazul «parcul bordei','verificarea ','ice dunăre'],
             "Külpolitika, diaszpóra":['din afara graniţelor','politică extern',
                                      'românii de pretutindeni','românilor de pretutindeni'],
             "Gazdaság, ipar":['economică','industrie şi servici','industrii şi servicii','ministerului economiei',
                              'pieţei de capital'],
             "Oktatás, tudomány, sport":['învăţământ', 'ştiinţă', 'tineret', 'sport','spaţiu'],
             "Egészségügy":['sănătate'],
             'Választások':['alegerilor','electoral'],
             "Biztonság":['ordinii publice']
            } 

    for a in abbr_in:
        for b in abbr_in[a]:
            if b in csop: return medal+biz_flags[a]+' '+a
    
    a='Más'
#     return medal+biz_flags[a]+' '+a
    return csop

In [283]:
# nmembers['Popescu-Tariceanu Calin Constantin Anton | 1952-01-14']

In [291]:
efs=[]
# for ni,n in enumerate(nmembers):
for ni,n in enumerate(['Popescu-Tariceanu Calin Constantin Anton | 1952-01-14']):
    if ni<5000:
        if ni%50==0: print(ni/len(nmembers)*100,'%')
        m=nmembers[n]
        data=[]
        since=m['Starts']
        until=m['Ends']
        for p in m['Parties']:
            start=pd.to_datetime(p['start'])
            end=pd.to_datetime(p['end'])    
            d={}
            d['Name']=m['Name']['full']
            party=p['party']
            if '#GRUP#' in party:
                party=group_party_normalizer(party)
            else:
                party=party_normalizer(p['party'])+' '+party_writer(p['party'])+' - '+img['party'][party_writer(p['party'])]['name']
            d['Csoportok']=party
            d['Típus']='Párt'
            d['Image']=img['party'][p['party']]['img']
            d['start']=start
            d['end']=end
            d['duration']=pd.to_timedelta(end-start).days
            data.append(d)
            d=dict(d)
            d['Csoportok']='🇷🇴 '+megyek[p['judet'].capitalize()]
            d['Típus']='Megye'
            data.append(d)
        for p in m['Countries']:
            start=pd.to_datetime(p['start'])
            end=pd.to_datetime(p['end'])    
            d={}
            d['Name']=m['Name']['full']
            d['Csoportok']=p['country']
            d['Típus']='Ország'
            d['Image']=img['state'][replace_all(p['country'],pres_all)]
            d['start']=start
            d['end']=end
            d['duration']=pd.to_timedelta(end-start).days
            data.append(d)
        for p in m['Groups']:
            start=pd.to_datetime(p['start'])
            end=pd.to_datetime(p['end'])    
            d={}
            d['Name']=m['Name']['full']
            if '#BP#' not in p['group']:
                csop=biz_cleaner(p['group'])
                if 'cu unesc' in csop:
                    d['Csoportok']='#INTER#UNESCO'
                    d['Típus']='Ország'
                    d['Image']=img['state']['Grupul parlamentar de prietenie cu UNESCO']
                elif 'și parlamentul republicii moldov' in csop:
                    d['Csoportok']='Moldova'
                    d['Típus']='Ország'
                    d['Image']='md'
                else:
                    d['Csoportok']=csop
                    d['Típus']='Bizottság'
            else:
                d['Csoportok']=p['group'][:-1]+szen[int(p['group'][-1])-1]
                d['Típus']='Kamara'
            d['start']=start
            d['end']=end
            d['duration']=pd.to_timedelta(end-start).days
            data.append(d)

        for pi,p in enumerate(m['Link']):
            start=pd.to_datetime(since[pi])
            end=pd.to_datetime(until[pi])
            d={}
            d['Name']=m['Name']['full']
            d['Típus']='Kamara'
            kamara=m['Camera'][pi]
            pres=''
#             if '#' in kamara:
#                 pres='#'+kamara.split('#')[1]+'#'
#                 if ('din ') in kamara:
#                     b=dict(d)
#                     b['Csoportok']=szen[int(p[-1])-1]
#                     b['start']=start
#                     start=max(start,date_ro(kamara[kamara.find('din ')+4:].strip(),False))
#                     b['end']=start
#                     b['duration']=pd.to_timedelta(end-start).days
#                     data.append(b)
#                 if ('până în ') in kamara:
#                     b=dict(d)
#                     b['Csoportok']=szen[int(p[-1])-1]
#                     b['end']=end
#                     end=min(end,date_ro(kamara[kamara.find('până în ')+8:].strip(),False))
#                     b['start']=end
#                     b['duration']=pd.to_timedelta(end-start).days
#                     data.append(b)
            d['Csoportok']=pres+szen[int(p[-1])-1]
            d['start']=start
            d['end']=end
            d['duration']=pd.to_timedelta(end-start).days
            data.append(d)

        ef=pd.DataFrame(data)
        ef=ef.set_index('Típus').reset_index()
        ef['Simple']=n

        #pres
        csv=ef['Csoportok'].values
        ef['Medals']=[replace_all(i[:i.rfind('#')+1],medals) if '#' in i else '' for i in csv]
        ef['Csoportok']=[replace_all(i,medals_none) for i in csv]

        #countries
        if 'Image' in ef.columns:
            cz={}
            cs=ef[ef['Típus']=='Ország'].groupby('Image').first()['Csoportok']
            for i in sorted(ef[ef['Típus']=='Ország']['Image'].unique()):
                if i in cc:
                    cz[cs.loc[i]]=local_country(cc[i])

            #manual overrides
            cz['Republica Macedonia']='🇲🇰 Macedónia'
            cz['Republica Federală Iugoslavia']='Jugoszlávia'
            cz['fosta Republică Iugoslavă Macedonia']='🇲🇰 Macedónia'
            cz['Grupul parlamentar de prietenie cu fosta Republică Iugoslavă Macedonia']='🇲🇰 Macedónia'
            cz['Republica Franceză - Senat']='🇫🇷 Franciaország'
            cz['Republica Serbia']='🇷🇸 Szerbia'
            cz['Republica Franceză - Adunarea Naţională']='🇫🇷 Franciaország'
            ef=ef.replace(cz).replace(cr)

        #matching
        ef['Csoportok9']=ef['Csoportok']
        ef['Csoportok']=ef['Medals']+ef['Csoportok']
        ef['Csoportok']=ef['Csoportok'].replace('','Más').replace('🥈','🥈 Más')\
            .replace('🥇','🥇 Más').replace('🥉','🥉 Más').replace('🏅','🏅 Más')

        #unique counting
        new_column=[]
        counters={i:0 for i in ef['Csoportok'].unique()}
        for i in ef['Csoportok']:
            new_column.append(i+counters[i]*' ')
            counters[i]+=1
        ef['Csoportok0']=new_column

        #tolerance
        ef=tolerance_cleaner(ef).reset_index()

        #append
        efs.append(ef)

0.0 %


In [292]:
efs2=pd.concat(efs)

In [293]:
efs4=efs2[['start', 'end', 'Name', 'duration',
       'Simple', 'Csoportok','Csoportok0','Csoportok1','Csoportok9','Típus','Medals']]
efs4['Medals']=efs4['Medals'].replace('',' ')
efs4=efs4[efs4['duration']>0]

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [294]:
efs4['Name']=efs4['Name'].str.replace('chestor al Camerei Deputaţilor','')\
    .str.replace('vicepreşedinte al Camerei Deputaţilor','')\
    .str.replace('secretar al Camerei Deputaţilor','')\
    .str.replace('Preşedintele Senatului','')\
    .str.replace('Preşedintele Camerei Deputaţilor','')\
    .str.replace('secretar al Senatului','')\
    .str.replace('chestor al Senatului','')\
    .str.replace('vicepreşedinte al Senatului','')

In [295]:
efs4[efs4['Simple']=='Popescu-Tariceanu Calin Constantin Anton | 1952-01-14']

,start,end,Name,duration,Simple,Csoportok,Csoportok0,Csoportok1,Csoportok9,Típus,Medals
0,1990-06-18,1992-10-16,Călin Constantin Anton POPESCU-TĂRICEANU,851,Popescu-Tariceanu Calin Constantin Anton | 195...,"⏫ Gazdaság, ipar","⏫ Gazdaság, ipar","⏫ Gazdaság, ipar","⏫ Gazdaság, ipar",Bizottság,
1,2008-12-19,2012-12-19,Călin Constantin Anton POPESCU-TĂRICEANU,1461,Popescu-Tariceanu Calin Constantin Anton | 195...,"🌐 Külpolitika, diaszpóra","🌐 Külpolitika, diaszpóra","🌐 Külpolitika, diaszpóra","🌐 Külpolitika, diaszpóra",Bizottság,
2,2012-12-20,2014-03-01,Călin Constantin Anton POPESCU-TĂRICEANU,436,Popescu-Tariceanu Calin Constantin Anton | 195...,"🌐 Külpolitika, diaszpóra","🌐 Külpolitika, diaszpóra","🌐 Külpolitika, diaszpóra","🌐 Külpolitika, diaszpóra",Bizottság,
3,1996-11-27,2000-11-30,Călin Constantin Anton POPESCU-TĂRICEANU,1464,Popescu-Tariceanu Calin Constantin Anton | 195...,🔀 Állami ügyek,🔀 Állami ügyek,🔀 Állami ügyek,🔀 Állami ügyek,Bizottság,
4,1990-06-18,1992-10-16,Călin Constantin Anton POPESCU-TĂRICEANU,851,Popescu-Tariceanu Calin Constantin Anton | 195...,"🔠 Oktatás, tudomány, sport","🔠 Oktatás, tudomány, sport","🔠 Oktatás, tudomány, sport","🔠 Oktatás, tudomány, sport",Bizottság,
5,2015-02-01,2016-12-20,Călin Constantin Anton POPESCU-TĂRICEANU,688,Popescu-Tariceanu Calin Constantin Anton | 195...,🚩🔀 Állami ügyek,🚩🔀 Állami ügyek,🚩🔀 Állami ügyek,🔀 Állami ügyek,Bizottság,🚩
6,2004-12-17,2008-12-13,Călin Constantin Anton POPESCU-TĂRICEANU,1457,Popescu-Tariceanu Calin Constantin Anton | 195...,"🚹 Emberi jogok, kisebbségek","🚹 Emberi jogok, kisebbségek","🚹 Emberi jogok, kisebbségek","🚹 Emberi jogok, kisebbségek",Bizottság,
7,2000-12-15,2004-11-30,Călin Constantin Anton POPESCU-TĂRICEANU,1446,Popescu-Tariceanu Calin Constantin Anton | 195...,🥈🔢 Pénzügy,🥈🔢 Pénzügy,🥈🔢 Pénzügy,🔢 Pénzügy,Bizottság,🥈
8,2008-12-19,2012-12-19,Călin Constantin Anton POPESCU-TĂRICEANU,1461,Popescu-Tariceanu Calin Constantin Anton | 195...,🥈🚩🔀 Állami ügyek,🥈🚩🔀 Állami ügyek,🥈🚩🔀 Állami ügyek,🔀 Állami ügyek,Bizottság,🥈🚩
9,2012-12-20,2016-12-20,Călin Constantin Anton POPESCU-TĂRICEANU,1461,Popescu-Tariceanu Calin Constantin Anton | 195...,🔼 Szenátus,🔼 Szenátus,🔼 Szenátus,🔼 Szenátus,Kamara,


In [296]:
j=efs4.reset_index().drop('index',axis=1).T.astype(str).to_dict()
open('ignore/member_timelines.json','w').write(json.dumps(list(j.values())))

29130

In [297]:
len(efs4['Simple'].unique())

1

In [298]:
import zipfile

with zipfile.ZipFile("json/member_timelines.zip", "w", compression=zipfile.ZIP_DEFLATED) as zf:
    zf.write("ignore/member_timelines.json","member_timelines.json")

In [1608]:
members={}
simple={}
simple2={}
for i in efs4[['Name','Simple']].T.iteritems():
    if i[1][0] not in simple: simple[i[1][0]]=i[1][1]
    if i[1][1] not in simple2: simple2[i[1][1]]=i[1][0]
members['s']=simple
members['s2']=simple2

In [1609]:
members['megye']=efs4[efs4['Típus']=='Megye'].sort_values(by='start')[['Simple','Csoportok']].set_index('Csoportok')\
    .drop_duplicates(keep='last').reset_index().set_index('Simple')['Csoportok'].to_dict()

In [1610]:
members['part']=efs4[efs4['Típus']=='Párt'].sort_values(by='start')[['Simple','Csoportok']].set_index('Csoportok')\
    .drop_duplicates(keep='last').reset_index().set_index('Simple')['Csoportok'].to_dict()

In [1611]:
members['deaths']=deaths

Vega direct plot

In [1612]:
m='Popescu-Tariceanu Calin Constantin Anton | 1952-01-14'
ef=efs4[efs4['Simple']==m]
viz=alt.Chart(ef,
          title=m
         ).mark_bar().encode(
    x=alt.X('start:T',title=''),
    x2='end:T',
    y=alt.Y('Csoportok:N',title='',
            sort=alt.EncodingSortField(field="order", op="mean", order='descending')),
    color=alt.Color('Típus:N',
        sort=alt.EncodingSortField(field="order", op="mean", order='descending'))
)
# viz.save('json/viz.json')
viz.interactive()

<VegaLite 3 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


Member images

In [1613]:
photos={name:nmembers[name]['Photo'][np.array([p[-10:-6] for p in nmembers[name]['Link']]).argmax()] for name in nmembers}

Download photos if you need to..

In [1221]:
import urllib.request

In [ ]:
#downloader - run once only
for p in photos:
    urllib.request.urlretrieve(photos[p], "photos3/"+p.replace('|','+')+'.jpg')

In [ ]:
from PIL import Image
#!pip install python-resize-image
from resizeimage import resizeimage

In [ ]:
err=[]
for p in photos:
    try:
        img = Image.open('photos3/'+p.replace('|','+')+'.jpg', 'r')
        img = resizeimage.resize_cover(img, [123,150])
        img.save('photos4/'+p.replace('|','+')+'.jpg', img.format)
    except:
        print(p)
        err.append(p)

In [ ]:
#run this then iterate cell above until no errors
for p in err:
    urllib.request.urlretrieve(photos[p], "photos3/"+p.replace('|','+')+'.jpg')

In [ ]:
for p in err:
    try:
        img = Image.open('photos3/'+p.replace('|','+')+'.jpg', 'r')
        img = resizeimage.resize_cover(img, [123,150])
        img.save('photos4/'+p.replace('|','+')+'.jpg', img.format)
    except:
        img = Image.open('photos3/'+p.replace('|','+')+'.jpg', 'r')
        img.save('photos4/'+p.replace('|','+')+'.jpg', img.format)

Otherwise continue here:

In [1614]:
members['photo']={p:'photos4/'+p.replace('|','+')+'.jpg' for p in photos}

In [1615]:
open('ignore/member_simple.json','w').write(json.dumps(members))

957435

In [1616]:
import zipfile

with zipfile.ZipFile("json/member_simple.zip", "w", compression=zipfile.ZIP_DEFLATED) as zf:
    zf.write("ignore/member_simple.json","member_simple.json")

In [1617]:
pretty={'- din care declaraţii politice':'🧾 Nyilatkozatok',
        'Declaraţii politice depuse în scris':'🧾 Nyilatkozatok',
       'Luari de cuvânt':'🎙️ Felszólalások', 
        'Luari de cuvânt în BP':'🎙️ Felszólalások', 
        'Motiuni':'📚 Javaslatok', 
        'Proiecte de hotarâre initiate':'📚 Javaslatok', 
        'Propuneri legislative initiate':'📚 Javaslatok',
        'Sedințe':'💺 Ülések', 
        'Întrebari si interpelari':'🎙️ Felszólalások'}

In [1618]:
efx=[]
for n in nmembers:
    m=nmembers[n]
    data=[]
    since=m['Starts']
    until=m['Ends']
    for pi in range(len(m['Link'])):
        p=m['Link'][pi]
        d={}
        d['Simple']=n
        d['Link']=p
        d['Session']=p[-10:-6]
        for a in m['Activity']:
            if type(m['Activity'][a])==list:
                g=m['Activity'][a]
            else:
                g=[m['Activity'][a]]
            for h in g:
                if h['start']==since[pi]: 
                    if pretty[a] not in d:d[pretty[a]]=0
                    d[pretty[a]]+=h['value']
        data.append(d)
    ef=pd.DataFrame(data)

    #append
    efx.append(ef)

In [1619]:
efx2=pd.concat(efx).fillna(0).reset_index().drop('index',axis=1)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [1620]:
j=efx2.T.astype(str).to_dict()
j2=list(j.values())

In [1621]:
j3={}
for i in j2:
    if i['Simple'] not in j3:j3[i['Simple']]=[]
    j3[i['Simple']].append(i)

In [1622]:
open('ignore/member_activity.json','w').write(json.dumps(j3))

1433467

In [1623]:
import zipfile

with zipfile.ZipFile("json/member_activity.zip", "w", compression=zipfile.ZIP_DEFLATED) as zf:
    zf.write("ignore/member_activity.json","member_activity.json")